In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from dataclasses import dataclass

In [2]:
current_dir = 'f:/buaa/python/final_proj'
os.chdir(current_dir)
sensor_file = 'data/mysenior_sensor_data_output.csv'
debris_file = 'data/reduced_debris_data.csv'
star_file = 'data/Star-100027_star.csv'
star_magnitude_file = 'data/Star-100027_magnitude.csv'
access_file_csv = 'data/Satellite-observe-Sensor-mysenior-To-Satellite-0_ALEXIS_22638_Access.csv'
access_file_txt = 'data/Satellite-observe-Sensor-mysenior-To-Satellite-0_ALEXIS_22638_Access.txt'
f = 0.01413
dh = 0.000006
dv = 0.000006
H = 700

In [4]:
class SpaceObject():
  def __init__(self, category, name='unknow'):
    self.name = name
    self.category = category # 'sensor', 'debris', 'star'

class Sensor(SpaceObject):
  def __init__(self, name, time, x, y, z, vx, vy, vz, q1, q2, q3, q4, wx, wy, wz, category='sensor'):
    super().__init__(category, name)
    self.time = pd.to_datetime(time)
    self.position = np.array([x, y, z])
    self.velocity = np.array([vx, vy, vz])
    self.quaternions = np.array([q1, q2, q3, q4])
    self.angular_velocity = np.array([wx, wy, wz])

class Star(SpaceObject):
  def __init__(self, name, magnitude, x, y, z, right_ascension, declination, category='star'):
    super().__init__(category, name)
    self.name = name
    self.magnitude = float(magnitude)
    self.position = np.array([x, y, z]).astype(float)
    self.as_dec = np.array([right_ascension, declination]).astype(float)

@dataclass()
class Debris:
  name: str
  time: pd.Timestamp
  magnitude: float
  position: np.ndarray

In [5]:
def read_sensor_data(sensor_file):
  sensor_data = pd.read_csv(sensor_file)
  time_list = pd.to_datetime(sensor_data['Time (UTCG)']).to_list()
  sensor=[]
  for i in range(len(sensor_data)):
    name = 'mysensor'
  for sensor_ in sensor_data.iterrows():
    sensor_temp = Sensor(name, sensor_[1]['Time (UTCG)'],sensor_[1]['x (km)'],sensor_[1]['y (km)'],sensor_[1]['z (km)'],sensor_[1]['Velocity x (km/sec)'],sensor_[1]['Velocity y (km/sec)'],sensor_[1]['Velocity z (km/sec)'],sensor_[1]['q1'],sensor_[1]['q2'],sensor_[1]['q3'],sensor_[1]['q4'],sensor_[1]['wx (deg/sec)'],sensor_[1]['wy (deg/sec)'],sensor_[1]['wz (deg/sec)'])
    sensor.append(sensor_temp)
  return sensor, time_list

def read_debris_data(debris_file):
  debris_data = pd.read_csv(debris_file,header=None)
  is_header = debris_data.iloc[:,0].str.startswith('Time (UTCG)')
  group_ids = is_header.cumsum()-1
  group_count = group_ids.max() + 1
  magnitudes = np.random.normal(4, 0.5, group_count)
  headers = debris_data[is_header].iloc[:, 2]
  names = headers.str.rsplit('-', n=1, expand=True)[0].str.replace(' ', '')
  debris_name_list = names.unique().tolist()
  debris = []
  for grp_id, sub_df in debris_data.groupby(group_ids):
    coords = sub_df.iloc[1:, 1:4].to_numpy(dtype=np.float64)
    time = pd.to_datetime(sub_df.iloc[1:, 0])
    name0 = names.iloc[grp_id]
    if name0.endswith('1') and (name0[:-1] in debris_name_list):
      debris_name_list.remove(name0[:-1])
      continue
    print(grp_id,names.iloc[grp_id])
    group_debris = [Debris(
        name = names.iloc[grp_id],
        time = time.iloc[i],
        magnitude = magnitudes[grp_id],
        position = coords[i]
      )for i in range(len(time))
    ]
    debris.append(group_debris)
  return debris, debris_name_list

def read_star_data(star_file,star_magnitude_file):
  star_data = pd.read_csv(star_file,header=None)
  magnitude_data  = pd.read_csv(star_magnitude_file,header=None)
  magnitude = [eval(magnitude_data.iloc[i][0]) for i in range(1,len(magnitude_data),2)]
  star = []
  star_name_list = []
  for _ in star_data.iterrows():
    if _[0]%3 == 0:
      name = _[1][1].rsplit('-',maxsplit=1)[0].replace(' ','')
      star_name_list.append(name)
    if _[0]%3 == 1:
      star_temp = Star(name, magnitude[_[0]//3], _[1][1], _[1][2], _[1][3], _[1][4], _[1][5])
      star.append(star_temp)
  return star, star_name_list

def read_access_data(access_file_csv,access_file_txt,skiprows=29):
  access = pd.read_csv(access_file_csv)
  access_df = pd.read_csv(access_file_txt,skiprows=skiprows,header=None)
  access_ls = []
  access = access.drop(list(range(1,len(access),2)))
  access.reset_index(drop=True,inplace=True)

  for i in range(len(access_df)):
    if access_df.iloc[i,0].replace(' ','').startswith('Access'):
      access_ls.append(access_df.iloc[i-2,0].split('-',maxsplit=2)[-1])
  access['name']=access_ls
  access.set_index('name',inplace=True)
  access['Start Time (UTCG)'] = pd.to_datetime(access['Start Time (UTCG)'])
  access['Stop Time (UTCG)'] = pd.to_datetime(access['Stop Time (UTCG)'])
  return access


In [6]:
sensor, time_list = read_sensor_data(sensor_file)
debris, debris_name_list = read_debris_data(debris_file)
star, star_name_list = read_star_data(star_file,star_magnitude_file)
access = read_access_data(access_file_csv,access_file_txt,29)

0 0_OPS_5721_09415
1 ASTRID_1_23464
2 COBE_20322
3 COSMOS_1176_11788
4 COSMOS_1181_11803
5 COSMOS_1302_12791
6 COSMOS_1357_13160
7 COSMOS_1431_13763
8 COSMOS_1476_14174
9 COSMOS_1637_15619
10 COSMOS_1930_18943
11 COSMOS_2008_19902
12 COSMOS_2270_23001
13 COSMOS_507_06120
14 COSMOS_617_06985
15 COSMOS_626_07005
16 COSMOS_645_07269
17 COSMOS_789_08591
18 DUMMY_MASS_1_24925
19 ESSA_9_03764
20 METEOR_1-15_06659
21 METEOR_1-23_08519
22 METEOR_PRIRODA_12585
23 SURCAL_159_02872
24 TIMATION_1_02847


In [ ]:
class DebrisInImage():#记录在图像中的碎片和星，包含坐标星等和灰度
  def __init__(self,name,time,u,v,x,y,z,magnitude,category):
    self.name = name
    self.time = time
    self.u = u
    self.v = v
    self.x = x
    self.y = y
    self.z = z
    self.magnitude = magnitude
    self.category = category
    max_magnitude_threshold = 7 #最大星等，也就是最暗的
    #self.gray = 255*2.512**(1-self.magnitude) #0黑 255白  ？
    self.gray = 255-255*2.512**(self.magnitude-7) #？
    #self.gray = 50+10*(6-self.magnitude)  #？
    self.gray = int(self.gray)
    self.gray = 255 if self.gray > 255 else self.gray
    self.gray = 0 if self.gray < 0 else self.gray


class SpaceImage():#绘制每个时刻的图像
  def __init__(self,time,f,dh,dv,H):
    self.time = time
    self.f = f
    self.dh = dh
    self.dv = dv
    self.H = H
    self.visible_debris=[]
    self.visible_star = []

  def rotate(self,sensor):
    self.sensor = sensor
    q = sensor.quaternions
    if q[3] < 0:
      q = -q
    q0, q1, q2, q3 = q
    Msi = np.array([[q3**2+q0**2-q1**2-q2**2, 2*q3*q2+2*q0*q1,     -2*q3*q1+2*q0*q2],
                    [-2*q3*q2+2*q0*q1,    q3**2-q0**2+q1**2-q2**2, 2*q3*q0+2*q1*q2],
                    [2*q3*q1+2*q0*q2,     -2*q3*q0+2*q1*q2,    q3**2-q0**2-q1**2+q2**2]])
    self.Msi = Msi
    return self.Msi

  def log_debris(self,debris0,debris_name):
    self.debris = debris0
    sensor_vec = self.sensor.position
    debris_vec = self.debris.position
    [x,y,z] = self.debris.position
    vec = sensor_vec-debris_vec
    distance = np.linalg.norm(vec)
    vec = vec/np.linalg.norm(vec)
    magnitude = self.debris.magnitude-3
    magnitude = magnitude -30 + 10*np.log10(distance)
    if magnitude > 7:
      return
    A = np.dot(self.Msi,vec)
    self.u = self.f*A[0]/(self.dh*A[2])
    self.v = self.f*A[1]/(self.dv*A[2])
    self.u_int, self.v_int = int(self.u), int(self.v)
    
    # print('u:',self.u," v:",self.v, 'magnitude:',magnitude)
    if self.u_int > self.H or self.u_int < -self.H or self.v_int > self.H or self.v_int < -self.H:
      print(self.u,self.v)
    debris_temp = DebrisInImage(debris_name,self.time,self.u,self.v,x,y,z,magnitude,'debris')
    self.visible_debris.append(debris_temp)

  def log_star(self,star,star_name):
    # 将位置矢量近似为方向矢量
    vec = star.position
    [x,y,z] = vec
    A = np.dot(self.Msi,vec)
    self.u = self.f*A[0]/(self.dh*A[2])
    self.v = self.f*A[1]/(self.dv*A[2])
    self.u_int, self.v_int = int(self.u), int(self.v)
    if self.u_int > self.H or self.u_int < -self.H or self.v_int > self.H or self.v_int < -self.H:
      print(self.u,self.v)
    star_temp = DebrisInImage(star_name,self.time,self.u,self.v,x,y,z, star.magnitude,'star')
    self.visible_star.append(star_temp)

  def diffuse_img(self, img, u, v, gray):
    sigma = 0.5  # 高斯核标准差
    radius = 2    # 对应5x5矩阵半径
    
    # 计算亚像素偏移后的中心坐标（相对于5x5矩阵）
    u_center = (u - int(u)) + radius
    v_center = (v - int(v)) + radius

    # 向量化生成高斯分布
    i, j = np.indices((5, 5))
    exponent = -((i - u_center)**2 + (j - v_center)**2) / (2 * sigma**2)
    diffuse = np.clip(gray * np.exp(exponent), 0, 255).astype(np.uint8)

    # 计算图像有效区域
    u_int, v_int = int(u), int(v)
    start_u, end_u = max(0, u_int - radius), min(img.shape[0], u_int + radius + 1)
    start_v, end_v = max(0, v_int - radius), min(img.shape[1], v_int + radius + 1)

    # 计算扩散矩阵的对应区域（自动处理边界裁剪）
    d_start_u = max(0, start_u - (u_int - radius))
    d_end_u = min(5, d_start_u + (end_u - start_u))
    d_start_v = max(0, start_v - (v_int - radius))
    d_end_v = min(5, d_start_v + (end_v - start_v))

    # 仅在有效区域进行赋值
    if d_start_u < d_end_u and d_start_v < d_end_v:
        img[start_u:end_u, start_v:end_v] = diffuse[d_start_u:d_end_u, d_start_v:d_end_v]
    
    return img

  def add_noise(self, img):
    noise = np.random.normal(10,10,(self.H*2,self.H*2))
    img = img + noise
    img_clipped = np.clip(img, 0, 255).astype(np.uint8)
    return img_clipped

  def plot_image(self,ifplot=False, ifsave=False, iflabel=False):
    # 662*662 焦距f=0.01413m  0.000006m/pixel, alpha = 16.05deg = 2arctan(H/2f), -->H=662
    visible_debris = self.visible_debris
    visible_star = self.visible_star
    img = np.zeros((self.H*2,self.H*2),dtype=np.uint8)
    for debris_ in visible_debris:
      img = SpaceImage.diffuse_img(self, img,debris_.u+self.H,debris_.v+self.H,debris_.gray)
      if iflabel:
        cv2.putText(img,debris_.name,(debris_.v+self.H+10,debris_.u+self.H+10),cv2.FONT_HERSHEY_SIMPLEX,0.2,(255,255,255),1)
    for star_ in visible_star:
      img = SpaceImage.diffuse_img(self, img,star_.u+self.H,star_.v+self.H,star_.gray)
      if iflabel:
        cv2.putText(img,star_.name,(star_.v+self.H+10,star_.u+self.H+10),cv2.FONT_HERSHEY_SIMPLEX,0.2,(255,255,255),1)
    img = SpaceImage.add_noise(self,img)
    if ifplot:
      cv2.imshow('image',img)
      cv2.waitKey(0)
    if ifsave:
      cv2.imwrite('image/'+"{:04d}".format(time_list.index(self.time))+'.png',img)
    return img

In [8]:
def images_sequence(star_index,end_index):
  images = []
  for curr_time in time_list[star_index:end_index]:
    time_index = time_list.index(curr_time)
    visible_star = []
    visible_debris = []
    #统计可见的星和碎片
    for i in range(len(access)):
      if access.iloc[i]['Start Time (UTCG)'] <= curr_time and access.iloc[i]['Stop Time (UTCG)'] >= curr_time:
        if access.index[i] in star_name_list:
          visible_star.append(access.index[i])
        if access.index[i] in debris_name_list:
          visible_debris.append(access.index[i])

    space_image = SpaceImage(curr_time, f, dh, dv,H)
    Msi = space_image.rotate(sensor[time_index])

    if len(visible_debris) != 0:
      for debris_name in visible_debris:
        debris_index = debris_name_list.index(debris_name)
        space_image.log_debris(debris[debris_index][time_index],debris_name)
    if len(visible_star) != 0:
      for star_name in visible_star:
        star_index = star_name_list.index(star_name)
        space_image.log_star(star[star_index],star_name)
    images.append(space_image)
  return images

In [11]:
images = images_sequence(0,len(time_list)-1)

In [12]:
log_data = pd.DataFrame(columns=['time','debris_num','star_num','target','x','y','z','u','v','magnitude','category'])
temp_log=[]
for image in images:
  print(image.time)
  image.plot_image(iflabel=False, ifsave=True, ifplot=False)
  visible_debris = [_.name for _ in image.visible_debris]
  visible_star = [_.name for _ in image.visible_star]
  for _ in image.visible_debris:
    ss_temp = {'time':image.time,'debris_num':len(image.visible_debris),'star_num':len(image.visible_star),'target':_.name,'x':_.x,'y':_.y,'z':_.z,'u':_.u,'v':_.v,'magnitude':_.magnitude,'category':_.category}
    temp_log.append(ss_temp)
  log_data = pd.concat([log_data,pd.DataFrame(temp_log)],ignore_index=True)
  temp_log = []
  for _ in image.visible_star:
    ss_temp = {'time':image.time,'debris_num':len(image.visible_debris),'star_num':len(image.visible_star),'target':_.name,'x':_.x,'y':_.y,'z':_.z,'u':_.u,'v':_.v,'magnitude':_.magnitude,'category':_.category}
    temp_log.append(ss_temp)
  log_data = pd.concat([log_data,pd.DataFrame(temp_log)],ignore_index=True)
cv2.destroyAllWindows()
log_data.to_csv('log_data.csv',index=False)

2023-04-25 16:00:00
2023-04-25 16:00:00.100000
2023-04-25 16:00:00.200000
2023-04-25 16:00:00.300000
2023-04-25 16:00:00.400000
2023-04-25 16:00:00.500000
2023-04-25 16:00:00.600000
2023-04-25 16:00:00.700000
2023-04-25 16:00:00.800000
2023-04-25 16:00:00.900000
2023-04-25 16:00:01
2023-04-25 16:00:01.100000
2023-04-25 16:00:01.200000
2023-04-25 16:00:01.300000
2023-04-25 16:00:01.400000
2023-04-25 16:00:01.500000
2023-04-25 16:00:01.600000
2023-04-25 16:00:01.700000
2023-04-25 16:00:01.800000
2023-04-25 16:00:01.900000
2023-04-25 16:00:02
2023-04-25 16:00:02.100000
2023-04-25 16:00:02.200000
2023-04-25 16:00:02.300000
2023-04-25 16:00:02.400000
2023-04-25 16:00:02.500000
2023-04-25 16:00:02.600000
2023-04-25 16:00:02.700000
2023-04-25 16:00:02.800000
2023-04-25 16:00:02.900000
2023-04-25 16:00:03
2023-04-25 16:00:03.100000
2023-04-25 16:00:03.200000
2023-04-25 16:00:03.300000
2023-04-25 16:00:03.400000
2023-04-25 16:00:03.500000
2023-04-25 16:00:03.600000
2023-04-25 16:00:03.700000
20